# This notebook is for IBM Data Scinse capstone project
## By Dmitrii Shabrat

# Neighbourhoods of Toronto

## Collecting data of Toronto neighbourhoods

In [92]:
#importing libraries
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Solving environment: ...working... 
  - anaconda/win-64::ca-certificates-2019.8.28-0, anaconda/win-64::openssl-1.1.1d-he774522_2
  - anaconda/win-64::openssl-1.1.1d-he774522_2, defaults/win-64::ca-certificates-2019.8.28-0
  - anaconda/win-64::ca-certificates-2019.8.28-0, defaults/win-64::openssl-1.1.1d-he774522_2
  - defaults/win-64::ca-certificates-2019.8.28-0, defaults/win-64::openssl-1.1.1d-he774522_2done

## Package Plan ##

  environment location: C:\Users\shabr\Anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          92 KB

The following NEW packages will be INSTALLED:



In [148]:
#scrapping wikipedia
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
nb=pd.DataFrame(df[0])

In [149]:
nb.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [49]:
nb['Neighbourhood'].value_counts()

Not assigned        78
Runnymede            2
St. James Town       2
Commerce Court       1
Woodbine Heights     1
                    ..
Ryerson              1
Silverthorn          1
Dorset Park          1
Island airport       1
Cedarbrae            1
Name: Neighbourhood, Length: 208, dtype: int64

In [53]:
nb['Borough'].value_counts()

Not assigned        77
Etobicoke           44
North York          38
Downtown Toronto    37
Scarborough         37
Central Toronto     17
West Toronto        13
York                 9
East Toronto         7
East York            6
Mississauga          1
Queen's Park         1
Name: Borough, dtype: int64

In [150]:
#dropping not assigned values and columns we dont need
indexes = nb.loc[nb['Neighbourhood']=='Not assigned'].index
nb.drop(indexes,inplace=True, axis=0)
nb.drop(columns='Postcode',inplace=True,axis=1)
nb.reset_index(inplace=True)
nb.drop(columns='index', inplace=True,axis=1)

In [151]:
nb.head()

,Borough,Neighbourhood
0,North York,Parkwoods
1,North York,Victoria Village
2,Downtown Toronto,Harbourfront
3,North York,Lawrence Heights
4,North York,Lawrence Manor


In [214]:
#function to obtain geolocation data
def get_location(nbrhd):
    geolocator = Nominatim(user_agent="t_explorer")
    location = geolocator.geocode(nbrhd)
    latitude = location.latitude
    longitude = location.longitude
    #print('The geograpical coordinate of smthn are {}, {}.'.format(latitude, longitude))
    return '{}'.format(latitude),'{}'.format(longitude)

In [224]:
#making lists to put them into get_location data
adresses=[]
newlist=[]
for bor,nbrhd in zip(nb['Borough'],nb['Neighbourhood']):
    label = '{}, Toronto'.format(nbrhd)
    adresses.append(label)
for bor,nbrhd in zip(nb['Borough'],nb['Neighbourhood']):
    newlist.append([bor,nbrhd])
newlist[0]

['North York', 'Parkwoods']

In [227]:
#getting geodata
locations=[]
for adress in adresses:
    try:
        loc = get_location(adress)
        locations.append(loc)
    except:
        try:
            loc = get_location(adress)
            locations.append(loc)
        except:
            NA = ['NA','NA']
            locations.append(NA)

In [229]:
#merging lists
for i in range(len(newlist)):
    newlist[i].append(locations[i][0])
    newlist[i].append(locations[i][1])

In [329]:
#preparing dataframe
toronto_data = pd.DataFrame(data=newlist,columns=['Borough','Neighborhood','Latitude','Longitude'])
toronto_data.replace(to_replace='NA',value=np.nan,inplace=True)
toronto_data.dropna(axis=0,inplace=True)
toronto_data.reset_index(drop=True,inplace=True)
toronto_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 4 columns):
Borough         93 non-null object
Neighborhood    93 non-null object
Latitude        93 non-null object
Longitude       93 non-null object
dtypes: object(4)
memory usage: 3.0+ KB


In [330]:
toronto_data['Latitude'] = pd.to_numeric(neighborhoods['Latitude'])
toronto_data['Longitude'] = pd.to_numeric(neighborhoods['Longitude'])

In [331]:
toronto_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 4 columns):
Borough         93 non-null object
Neighborhood    93 non-null object
Latitude        93 non-null float64
Longitude       93 non-null float64
dtypes: float64(2), object(2)
memory usage: 3.0+ KB


In [333]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_data['Borough'].unique()),
        geodata.shape[0]
    )
)

The dataframe has 9 boroughs and 93 neighborhoods.


In [334]:
toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,North York,Parkwoods,43.758800,-79.320197
1,North York,Victoria Village,43.732658,-79.311189
2,Downtown Toronto,Harbourfront,43.640080,-79.380150
3,North York,Lawrence Heights,43.722778,-79.450933
4,Downtown Toronto,Queen's Park,43.659659,-79.390340


## Collecting venues data

In [338]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [339]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [317]:
CLIENT_ID = '2J2XW3UMQSA4HQVAEHSPF4S3U0KJCIZ4PSGEGKBBDARFRABX'
CLIENT_SECRET = 'OZIL1STXTN4H3GUDXKCAXRK2NDKSP1DJANQPIIS0PPNTNKWN'
VERSION = '20200225' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2J2XW3UMQSA4HQVAEHSPF4S3U0KJCIZ4PSGEGKBBDARFRABX
CLIENT_SECRET:OZIL1STXTN4H3GUDXKCAXRK2NDKSP1DJANQPIIS0PPNTNKWN


In [340]:
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7587999, -79.32019659999999.


In [318]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [341]:
radius = 500
LIMIT = 100
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront
Lawrence Heights
Queen's Park
Glencairn
Cloverdale
Princess Gardens
West Deane Park
Flemingdon Park
Don Mills South
Woodbine Heights
Old Burnhamthorpe
Morningside
West Hill
The Beaches
Berczy Park
Cedarbrae
Hillcrest Village
Wilson Heights
Thorncliffe Park
Dovercourt Village
Scarborough Village
Henry Farm
York University
Harbourfront East
Little Portugal
Trinity
Downsview East
Design Exchange
Toronto Dominion Centre
Exhibition Place
Parkdale Village
Clairlea
Silver Hills
India Bazaar
Commerce Court
Victoria Hotel
Downsview
North Park
Upwood Park
Humber Summit
Cliffcrest
Newtonbrook
Studio District
Lawrence Manor East
Silverthorn
Emery
Humberlea
Downsview Northwest
Lawrence Park
Scarborough Town Centre
Davisville North
Forest Hill North
High Park
Wexford
Willowdale West
North Midtown
Yorkville
Parkdale
Roncesvalles
Martin Grove Gardens
Richview Gardens
Agincourt
Harbord
Swansea
Sullivan
Moore Park
Chinatown
Grange Park
Agincourt North
L'Amoreaux E

In [342]:
print(toronto_venues.shape)
toronto_venues.head()

(2918, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7588,-79.320197,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.7588,-79.320197,LCBO,43.757774,-79.314257,Liquor Store
2,Parkwoods,43.7588,-79.320197,Petro-Canada,43.757950,-79.315187,Gas Station
3,Parkwoods,43.7588,-79.320197,Shoppers Drug Mart,43.760857,-79.324961,Pharmacy
4,Parkwoods,43.7588,-79.320197,TD Canada Trust,43.757569,-79.314976,Bank


In [343]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,11,11,11,11,11,11
Agincourt North,26,26,26,26,26,26
Albion Gardens,13,13,13,13,13,13
Bathurst Quay,24,24,24,24,24,24
Berczy Park,100,100,100,100,100,100
Cedarbrae,23,23,23,23,23,23
Chinatown,88,88,88,88,88,88
Church and Wellesley,85,85,85,85,85,85
Clairlea,9,9,9,9,9,9


In [344]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 297 uniques categories.


## Preparing data for clustering

In [404]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-101]] + list(toronto_onehot.columns[:-101])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Service,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [405]:
toronto_onehot.shape

(2918, 197)

In [406]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Service,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Hockey Field,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.090909,0.000000,0.000000,0.272727,0.000000,0.000000,0.00,0.000000,0.000000,0.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.0000

In [407]:
toronto_grouped.shape

(92, 197)

In [408]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.27
1  Cantonese Restaurant  0.09
2           Coffee Shop  0.09
3  Hong Kong Restaurant  0.09
4      Asian Restaurant  0.09


----Agincourt North----
                  venue  freq
0                Bakery  0.08
1    Chinese Restaurant  0.08
2  Fast Food Restaurant  0.08
3     Indian Restaurant  0.04
4        Ice Cream Shop  0.04


----Albion Gardens----
                  venue  freq
0         Grocery Store  0.15
1            Beer Store  0.08
2  Caribbean Restaurant  0.08
3  Fast Food Restaurant  0.08
4        Hardware Store  0.08


----Bathurst Quay----
             venue  freq
0      Coffee Shop  0.17
1             Café  0.12
2  Harbor / Marina  0.04
3             Bank  0.04
4     Dance Studio  0.04


----Berczy Park----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.06
2                Hotel  0.04
3               Bakery  0.04
4  Japanese Restaurant  0.04


----Cedarbrae----
 

                  venue  freq
0           Coffee Shop  0.08
1     Korean Restaurant  0.08
2        Discount Store  0.05
3        Hardware Store  0.05
4  Fast Food Restaurant  0.05


----North Midtown----
                  venue  freq
0  Caribbean Restaurant  0.08
1          Hockey Field  0.08
2          Intersection  0.08
3                  Café  0.08
4    Italian Restaurant  0.08


----North Park----
                 venue  freq
0   Chinese Restaurant  0.25
1  American Restaurant  0.25
2          Coffee Shop  0.25
3    Convenience Store  0.25
4           Hobby Shop  0.00


----Old Burnhamthorpe----
            venue  freq
0         Dog Run   0.4
1     Flower Shop   0.2
2     Gas Station   0.2
3  History Museum   0.0
4        Gym Pool   0.0


----Old Mill North----
                  venue  freq
0         Metro Station  0.14
1   American Restaurant  0.14
2     Accessories Store  0.00
3  Gym / Fitness Center  0.00
4              Gym Pool  0.00


----Old Mill South----
                  v

In [409]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [410]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Cantonese Restaurant,Hong Kong Restaurant,Coffee Shop,Korean Restaurant,Asian Restaurant,Comedy Club,Costume Shop,Cosmetics Shop,Convention Center
1,Agincourt North,Fast Food Restaurant,Chinese Restaurant,Bakery,Frozen Yogurt Shop,Liquor Store,Bank,Dim Sum Restaurant,Convenience Store,Fried Chicken Joint,Discount Store
2,Albion Gardens,Grocery Store,Fast Food Restaurant,Fried Chicken Joint,Gym Pool,Hardware Store,Beer Store,Caribbean Restaurant,Clothing Store,Chiropractor,Cosmetics Shop
3,Bathurst Quay,Coffee Shop,Café,Caribbean Restaurant,Gym,Dance Studio,Harbor / Marina,Bank,Garden,Diner,Japanese Restaurant
4,Berczy Park,Coffee Shop,Café,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Beer Bar,Gym,Cheese Shop,Creperie


## Clustering

In [411]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 0, 0, 1, 1, 1, 0, 1, 0, 0])

In [412]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,Parkwoods,43.758800,-79.320197,1.0,Laundry Service,Discount Store,Caribbean Restaurant,Chinese Restaurant,Liquor Store,Convenience Store,Coffee Shop,Bank,Gas Station,Cosmetics Shop
1,North York,Victoria Village,43.732658,-79.311189,0.0,Bus Line,Food Service,Middle Eastern Restaurant,Nail Salon,College Theater,Costume Shop,Cosmetics Shop,Convention Center,Convenience Store,Construction & Landscaping
2,Downtown Toronto,Harbourfront,43.640080,-79.380150,1.0,Coffee Shop,Café,Hotel,Italian Restaurant,Music Venue,History Museum,Gym,Brewery,Fried Chicken Joint,Chinese Restaurant
3,North York,Lawrence Heights,43.722778,-79.450933,0.0,Clothing Store,Coffee Shop,Cosmetics Shop,Electronics Store,Men's Store,Furniture / Home Store,American Restaurant,Jewelry Store,Fast Food Restaurant,Food Court
4,Downtown Toronto,Queen's Park,43.659659,-79.390340,1.0,Coffee Shop,Café,Italian Restaurant,French Restaurant,Juice Bar,Chinese Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Ice Cream Shop


In [413]:
toronto_merged.dropna(axis=0,inplace=True)

In [414]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [415]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Victoria Village,Bus Line,Food Service,Middle Eastern Restaurant,Nail Salon,College Theater,Costume Shop,Cosmetics Shop,Convention Center,Convenience Store,Construction & Landscaping
3,Lawrence Heights,Clothing Store,Coffee Shop,Cosmetics Shop,Electronics Store,Men's Store,Furniture / Home Store,American Restaurant,Jewelry Store,Fast Food Restaurant,Food Court
5,Glencairn,Asian Restaurant,Grocery Store,Japanese Restaurant,Metro Station,College Arts Building,Convention Center,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop
9,Flemingdon Park,Café,Movie Theater,Japanese Restaurant,Gym,Grocery Store,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Convenience Store,Construction & Landscaping
11,Woodbine Heights,Athletics & Sports,Dance Studio,Bus Stop,Cosmetics Shop,College Theater,Costume Shop,Convention Center,Convenience Store,Construction & Landscaping,Concert Hall
12,Old Burnhamthorpe,Dog Run,Flower Shop,Gas Station,Nail Salon,Costume Shop,Cosmetics Shop,Convention Center,Convenience Store,Construction & Landscaping,Concert Hall
13,Morningside,Fast Food Restaurant,Arts & Crafts Store,Health & Beauty Service,Big Box Store,Cosmetics Shop,Caribbean Restaurant,Ethiopian Restaurant,Empanada Restaurant,Convention Center,Convenience Store
14,West Hill,Fast Food Restaurant,Grocery Store,Breakfast Spot,Coffee Shop,Bus Line,Medical Center,Beer Store,Laundromat,Juice Bar,Greek Restaurant
15,The Beaches,Beach,Japanese Restaurant,Bakery,Bar,Breakfast Spot,Coffee Shop,BBQ Joint,Martial Arts Dojo,Grocery Store,Diner
18,Hillcrest Village,Ice Cream Shop,Grocery Store,Bank,Korean Restaurant,Coffee Shop,College Theater,Convention Center,Convenience Store,Construction & Landscaping,Concert Hall


In [416]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Parkwoods,Laundry Service,Discount Store,Caribbean Restaurant,Chinese Restaurant,Liquor Store,Convenience Store,Coffee Shop,Bank,Gas Station,Cosmetics Shop
2,Harbourfront,Coffee Shop,Café,Hotel,Italian Restaurant,Music Venue,History Museum,Gym,Brewery,Fried Chicken Joint,Chinese Restaurant
4,Queen's Park,Coffee Shop,Café,Italian Restaurant,French Restaurant,Juice Bar,Chinese Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Bubble Tea Shop,Ice Cream Shop
6,Cloverdale,Fast Food Restaurant,Coffee Shop,Camera Store,Department Store,Liquor Store,Baseball Field,Concert Hall,Gas Station,Garden Center,Furniture / Home Store
7,Princess Gardens,Coffee Shop,Aquarium,Hotel,Baseball Stadium,Gym,Grocery Store,History Museum,Indian Restaurant,Bar,Fast Food Restaurant
10,Don Mills South,Clothing Store,Fast Food Restaurant,Coffee Shop,Japanese Restaurant,Food Court,Juice Bar,Bakery,Gift Shop,Cosmetics Shop,Boutique
16,Berczy Park,Coffee Shop,Café,Hotel,Japanese Restaurant,Italian Restaurant,Bakery,Beer Bar,Gym,Cheese Shop,Creperie
17,Cedarbrae,Fast Food Restaurant,Clothing Store,Coffee Shop,Hardware Store,Big Box Store,Liquor Store,Grocery Store,Gym,Furniture / Home Store,Beer Store
20,Thorncliffe Park,Indian Restaurant,Afghan Restaurant,Coffee Shop,Fast Food Restaurant,Burger Joint,Gym,Liquor Store,Gas Station,Bank,Fried Chicken Joint
21,Dovercourt Village,Café,Bus Line,Bar,Coffee Shop,Brazilian Restaurant,Nail Salon,Comedy Club,Costume Shop,Cosmetics Shop,Convention Center


In [417]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Richview Gardens,Gym,Intersection,Home Service,Nail Salon,College Auditorium,Convention Center,Convenience Store,Construction & Landscaping,Concert Hall,Comic Shop


In [418]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,York University,Martial Arts Dojo,Gas Station,Nail Salon,Cuban Restaurant,Costume Shop,Cosmetics Shop,Convention Center,Convenience Store,Construction & Landscaping,Concert Hall


In [419]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,North Park,Chinese Restaurant,American Restaurant,Convenience Store,Coffee Shop,Nail Salon,Comedy Club,Creperie,Costume Shop,Cosmetics Shop,Convention Center
63,Agincourt,Chinese Restaurant,Cantonese Restaurant,Hong Kong Restaurant,Coffee Shop,Korean Restaurant,Asian Restaurant,Comedy Club,Costume Shop,Cosmetics Shop,Convention Center
71,L'Amoreaux East,Chinese Restaurant,Bus Stop,Coffee Shop,Athletics & Sports,Nail Salon,Comedy Club,Costume Shop,Cosmetics Shop,Convention Center,Convenience Store
79,L'Amoreaux West,Chinese Restaurant,Bus Stop,Coffee Shop,Athletics & Sports,Nail Salon,Comedy Club,Costume Shop,Cosmetics Shop,Convention Center,Convenience Store


So we got 2 main clusters wich i would define as fancy and regular neighborhoods.